In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def solve_cosine_potential_band_structure(a=4.0, V0=0.3, ngx=20, nbnds=5):

    n_vals = np.arange(-ngx, ngx + 1)
    G_vecs = n_vals * (2 * np.pi / a)
    U_coupling = V0 / 2.0
    k_norm = np.linspace(-0.5, 0.5, 200)
    k_vals = k_norm * (2 * np.pi / a)
    
    energies = []
    
    for k in k_vals:
        H_diag = 0.5 * (k - G_vecs)**2
        H_mat = np.diag(H_diag)
        np.fill_diagonal(H_mat[1:], U_coupling)
        np.fill_diagonal(H_mat[:, 1:], U_coupling)
        vals = np.linalg.eigvalsh(H_mat)
        vals = np.sort(vals)
        energies.append(vals[:nbnds])
        
    return k_norm, np.array(energies)

a_param = 4.0
V0_param = 0.3
k_axis, E_bands = solve_cosine_potential_band_structure(a=a_param, V0=V0_param)
plt.figure(figsize=(6, 5), dpi=150)

for i in range(E_bands.shape[1]):
    plt.plot(k_axis, E_bands[:, i], lw=2, label=f'Band {i+1}')

plt.title(f'Band Structure for $V(x) = {V0_param} \cos(2\pi x/{a_param})$', fontsize=14)
plt.ylabel('Energy (Atomic Units)', fontsize=12)
plt.xlabel(r'Wavevector $k$ ($\frac{2\pi}{a}$)', fontsize=12)
plt.xticks([-0.5, 0, 0.5], [r'$-\pi/a$', r'$0$', r'$\pi/a$'])
plt.axvline(0.5, color='gray', linestyle=':', alpha=0.5)
plt.axvline(-0.5, color='gray', linestyle=':', alpha=0.5)

plt.grid(True, alpha=0.3)
plt.xlim(-0.5, 0.5)

gap_low = E_bands[-1, 0]
gap_high = E_bands[-1, 1]
plt.annotate(f'Band Gap', xy=(0.5, (gap_low+gap_high)/2), xytext=(0.2, (gap_low+gap_high)/2),
             arrowprops=dict(facecolor='black', arrowstyle='->'))

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import scipy.sparse as spa
from scipy.sparse.linalg import eigsh

L = 600.0
N = 2048
x = np.linspace(-L/2, L/2, N)
dx = x[1] - x[0]

a = 4.0
v0 = 0.3

k0 = 0.0
sigma_k = 0.04
n0 = 0


def epsilon_nk_cosx_pot_tdse(k, a, v0, ngx=20, nbnds=5):
    Gx = np.arange(ngx, dtype=int)
    Gx[ngx // 2 + 1:] -= ngx
    b = 2 * np.pi / a
    Gx = Gx * b
    k_val = k * b

    lambda_kG = 0.5 * (k_val + Gx)**2
    Hij = np.diag(lambda_kG)

    coupling = v0 / 2.0
    
    nn = range(ngx)
    for ii in range(1, 2): 
        Hij[nn[:-ii], nn[ii:]] = coupling
        Hij[nn[:-(ngx-ii)], nn[ngx-ii:]] = coupling
        Hij[nn[ii:], nn[:-ii]] = coupling
        Hij[nn[(ngx-ii):], nn[:-(ngx-ii)]] = coupling
    
    E, phi = np.linalg.eigh(Hij) 
    return E[:nbnds], phi[:,:nbnds].T

def get_gaussian_envelope(k0, sigma_k, nk=100, Nsigma=6):
    delta_k = Nsigma * sigma_k
    ktmp = np.linspace(0, delta_k, nk, endpoint=True)
    k_rel = np.r_[-ktmp[::-1][:-1], ktmp]
    
    k_points = k_rel + k0
    
    k_points[k_points >= 0.5] -= 1.0
    k_points[k_points < -0.5] += 1.0
    
    weights = np.exp(-k_rel**2 / (2 * sigma_k**2))
    weights /= np.sqrt(np.sum(weights**2))

    return k_points, weights

def construct_bloch_packet_tdse(x, k_points, weights, n0, a, v0, ngx=20):
    Nx = x.size
    b = 2 * np.pi / a
    bloch_wp = np.zeros(Nx, dtype=complex)
    
    for ik, k in enumerate(k_points):
        _, C_matrix = epsilon_nk_cosx_pot_tdse(k, a, v0, ngx=ngx, nbnds=n0+1)
        C_G = C_matrix[n0]
        
        Gx_int = np.arange(ngx, dtype=int)
        Gx_int[ngx // 2 + 1:] -= ngx
        G_vecs = Gx_int * b
        
        psi_k = np.zeros(Nx, dtype=complex)
        real_k = k * b
        
        for ig, G in enumerate(G_vecs):
            psi_k += C_G[ig] * np.exp(1j * (real_k + G) * x)
            
        bloch_wp += weights[ik] * psi_k
        
    norm = np.sqrt(np.sum(np.abs(bloch_wp)**2) * dx)
    bloch_wp /= norm
    
    return bloch_wp

if __name__ == "__main__":
    kwp, fk = get_gaussian_envelope(k0, sigma_k, nk=150, Nsigma=8)
    
    k_bz = np.linspace(-0.5, 0.5, 200)
    nbnds_plot = 3
    Enk = np.array([epsilon_nk_cosx_pot_tdse(k, a, v0, nbnds=nbnds_plot)[0] for k in k_bz])
    
    psi_real = construct_bloch_packet_tdse(x, kwp, fk, n0, a, v0)

    import matplotlib as mpl
    mpl.rcParams['axes.unicode_minus'] = False
    plt.style.use('dark_background')

    fig = plt.figure(figsize=(10, 4), dpi=150)
    
    gs = fig.add_gridspec(1, 2, width_ratios=[1, 3])
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])

    for i in range(nbnds_plot):
        ax1.plot(k_bz, Enk[:, i], color='white', lw=1.5, alpha=0.8)
    
    emin = Enk[:, 0].min()
    band_width = Enk[:, 0].max() - emin
    scaled_fk = fk / fk.max() * (band_width * 0.5)
    
    e_k0 = epsilon_nk_cosx_pot_tdse(k0, a, v0, nbnds=1)[0][0]
    
    ax1.fill_between(kwp, e_k0, e_k0 + scaled_fk, color='cyan', alpha=0.6, label='Wavepacket (k)')
    
    ax1.set_xlim(-0.5, 0.5)
    ax1.set_xticks([-0.5, 0, 0.5])
    ax1.set_xticklabels([r'-$\pi/a$', '0', r'$\pi/a$'])
    ax1.set_xlabel('Wavevector k')
    ax1.set_ylabel('Energy (a.u.)')
    ax1.set_title('Band Structure & k-dist')
    ax1.axvline(0, color='gray', ls='--', lw=0.5)

    ax2.plot(x, np.abs(psi_real), color='gold', lw=1.2, label=r'$|\psi(x)|$')
    v_plot = (np.cos(2*np.pi*x/a) * v0)
    v_plot_scaled = v_plot / v0 * (np.max(np.abs(psi_real))*0.2) 
    ax2.plot(x, v_plot_scaled - np.max(np.abs(psi_real))*0.1, color='white', alpha=0.1, ls='--', label='Lattice Potential')

    ax2.set_xlim(-150, 150)
    ax2.set_xlabel('Position x (a.u.)')
    ax2.set_ylabel('Amplitude')
    ax2.set_title(f'Bloch Wavepacket in Real Space (a={a}, V0={v0})')
    ax2.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

In [ ]:
#!/usr/bin/env python

import os
import numpy as np
import scipy.sparse as spa
from scipy.sparse.linalg import splu
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.ticker import AutoMinorLocator

def CrankNicolson(psi0, V_diag, x, dt, N_steps=100):
    J  = x.size
    dx = x[1] - x[0]

    V_mat = spa.diags(V_diag)
    
    ones = np.ones(J)
    T_mat = (-0.5 / dx**2) * spa.spdiags([ones, -2*ones, ones], [-1, 0, 1], J, J)

    H = T_mat + V_mat

    lhs = spa.eye(J) + (1j * 0.5 * dt) * H
    rhs = spa.eye(J) - (1j * 0.5 * dt) * H
    
    lhs = lhs.tocsc()
    LU = splu(lhs)

    PSI_t = np.zeros((J, N_steps), dtype=complex)
    psi_current = psi0.copy()
    PSI_t[:, 0] = psi_current

    print(f"Propagating for {N_steps} steps...")
    for n in range(N_steps - 1):
        b = rhs.dot(psi_current)
        psi_current = LU.solve(b)
        PSI_t[:, n+1] = psi_current

    return PSI_t

def epsilon_nk_cosx_pot(k, a, v0, ngx=20, nbnds=5):
    Gx = np.arange(ngx, dtype=int)
    Gx[ngx // 2 + 1:] -= ngx
    b = 2 * np.pi / a
    
    G_vec = Gx * b
    k_val = k * b 

    lambda_kG = 0.5 * (k_val + G_vec)**2
    Hij = np.diag(lambda_kG)

    coupling = v0 / 2.0
    nn = range(ngx)
    
    for ii in range(1, 2):
        Hij[nn[:-ii], nn[ii:]] = coupling
        Hij[nn[:-(ngx-ii)], nn[ngx-ii:]] = coupling
        Hij[nn[ii:], nn[:-ii]] = coupling
        Hij[nn[(ngx-ii):], nn[:-(ngx-ii)]] = coupling
    
    E, phi = np.linalg.eigh(Hij) 
    return E[:nbnds], phi[:,:nbnds].T

def get_bloch_wavepacket_gaussian_envelop(k0, sigma_k, nk=100, Nsigma=6):
    delta_k = Nsigma * sigma_k
    ktmp = np.linspace(0, delta_k, nk, endpoint=True)
    k_rel = np.r_[-ktmp[::-1][:-1], ktmp]
    
    k_points = k_rel + k0
    
    k_points[k_points >= 0.5] -= 1.0
    k_points[k_points < -0.5] += 1.0
    
    weights = np.exp(-k_rel**2 / (2 * sigma_k**2))
    weights /= np.sqrt(np.sum(weights**2))
    
    return k_points, weights

def construct_blochwp_tdse(x, k_points, weights, n0, a, v0, ngx=20):
    Nx = x.size
    b = 2 * np.pi / a
    bloch_wp = np.zeros(Nx, dtype=complex)
    
    for ik, k in enumerate(k_points):
        _, C_matrix = epsilon_nk_cosx_pot(k, a, v0, ngx=ngx, nbnds=n0+1)
        C_G = C_matrix[n0]
        
        Gx_int = np.arange(ngx, dtype=int)
        Gx_int[ngx // 2 + 1:] -= ngx
        G_vecs = Gx_int * b
        
        real_k = k * b
        psi_k = np.zeros(Nx, dtype=complex)
        for ig, G in enumerate(G_vecs):
            psi_k += C_G[ig] * np.exp(1j * (real_k + G) * x)
            
        bloch_wp += weights[ik] * psi_k
        
    norm = np.sqrt(np.trapz(np.abs(bloch_wp)**2, x))
    return bloch_wp / norm


if __name__ == "__main__":
    L  = 600.0          # Space size
    N  = 2048           # Grid points
    x  = np.linspace(-L/2, L/2, N)
    dx = x[1] - x[0]
    
    a  = 4.0            # Lattice constant
    v0 = 0.3            # Potential depth
    b  = 2 * np.pi / a

    n0      = 0         # Band index
    k0      = 0.0       # Center momentum
    sigma_k = 0.04      # Momentum spread
    ngx     = 21        # No. of plane waves
    
    kwp, fk = get_bloch_wavepacket_gaussian_envelop(k0, sigma_k, nk=100)
    psi0 = construct_blochwp_tdse(x, kwp, fk, n0, a, v0, ngx)

    F_field = 0.005     # Electric field strength (small enough for Bloch osc.)
    
    V_x = v0 * np.cos(b * x) - F_field * x
    
    T_bloch = 2 * np.pi / (a * F_field)
    dt = 0.5
    total_time = 1.5 * T_bloch  # Simulate 1.5 periods
    NSW = int(total_time / dt)  # Number of steps

    print(f"Bloch Period: {T_bloch:.2f} a.u.")
    print(f"Total Steps: {NSW}")

    PSI_t = CrankNicolson(psi0, V_x, x, dt, NSW)

    prob_density = np.abs(PSI_t)**2
    norm_t = np.sum(prob_density, axis=0) * dx
    X_avg = np.sum(prob_density * x[:, None], axis=0) * dx / norm_t
    
    k_bz = np.linspace(-0.5, 0.5, 100)
    Enk = np.array([epsilon_nk_cosx_pot(k, a, v0, nbnds=3)[0] for k in k_bz])
    
    K_center_t = k0 - (F_field * np.arange(NSW) * dt) * (a / (2*np.pi))
    K_center_t = (K_center_t + 0.5) % 1.0 - 0.5

    mpl.rcParams['axes.unicode_minus'] = False
    plt.style.use('dark_background')

    fig = plt.figure(figsize=(10, 4), dpi=150)
    gs = fig.add_gridspec(1, 2, width_ratios=[1, 3])
    ax_band = fig.add_subplot(gs[0])
    ax_real = fig.add_subplot(gs[1])
    ax_pot = ax_real.twinx() # For Potential plotting

    for i in range(3):
        ax_band.plot(k_bz, Enk[:, i], 'w-', lw=1)
    
    scat_k = ax_band.scatter([], [], c='r', s=50, zorder=5)
    
    ax_band.set_xlim(-0.5, 0.5)
    ax_band.set_xticks([-0.5, 0, 0.5])
    ax_band.set_xticklabels([r'-$\pi/a$', '0', r'$\pi/a$'])
    ax_band.set_title('Momentum Space')
    ax_band.set_ylabel('Energy (a.u.)')

    line_psi, = ax_real.plot([], [], 'r-', lw=1.5, label=r'$|\psi|^2$')
    line_xavg, = ax_real.plot([], [], 'wo', ms=5, label=r'$\langle x \rangle$')
    
    mask = (x > -100) & (x < 100)
    ax_pot.plot(x[mask], V_x[mask], 'w--', lw=0.5, alpha=0.3)
    
    ax_real.set_xlim(-100, 100) # Zoom in to see oscillation
    ax_real.set_ylim(0, np.max(np.abs(psi0)**2)*1.2)
    ax_real.set_xlabel('Position (a.u.)')
    ax_real.set_title('Real Space Dynamics')
    
    time_text = ax_real.text(0.05, 0.9, '', transform=ax_real.transAxes, color='white')

    def init():
        scat_k.set_offsets(np.empty((0, 2)))
        line_psi.set_data([], [])
        line_xavg.set_data([], [])
        time_text.set_text('')
        return scat_k, line_psi, line_xavg, time_text

    def update(frame):
        idx = frame * 5 
        if idx >= NSW: idx = NSW - 1
        k_curr = K_center_t[idx]
        k_idx = np.argmin(np.abs(k_bz - k_curr))
        e_curr = Enk[k_idx, 0]
        scat_k.set_offsets([[k_curr, e_curr]])
        
        line_psi.set_data(x, np.abs(PSI_t[:, idx])**2)
        line_xavg.set_data([X_avg[idx]], [0])
        
        t_curr = idx * dt
        time_text.set_text(f't = {t_curr:.1f} a.u.\n({t_curr/T_bloch:.2f} $T_B$)')
        
        return scat_k, line_psi, line_xavg, time_text

    ani = animation.FuncAnimation(fig, update, frames=NSW//5, init_func=init, blit=True, interval=30)
    
    ani.save('bloch_oscillation_tdse.mp4', writer='ffmpeg', dpi=300)
    plt.show()